In [1]:
from ultralytics import YOLO
from PIL import Image
import cv2

# Load the YOLOv8 model
model = YOLO("runs/barrel/yolov8n_custom/weights/best.pt", task="detection")

In [2]:
# Define the path or URL to the image
image_path = "datasets/images/val/barrel/image_0_0.jpg"

# Run inference on the image
results = model(image_path)

# Get the first result (assuming only one image was passed)
result = results[0]

# Visualize the detections on the image
result_image = result.plot()

# Convert the color space from BGR to RGB
rgb_image = result_image[:, :, ::-1]

# Display the image with detections
Image.fromarray(rgb_image).show()


image 1/1 c:\Users\dk\Desktop\barrel_detection\datasets\images\val\barrel\image_0_0.jpg: 640x640 1 barrel, 8.1ms
Speed: 4.0ms preprocess, 8.1ms inference, 80.2ms postprocess per image at shape (1, 3, 640, 640)


In [3]:
cv2.imwrite('detected_image.jpg', result_image)

True

In [4]:
# Get the bounding box coordinates and confidence score of the detected barrel
boxes = result.boxes
conf = boxes.conf
xyxy = boxes.xyxy

# Extract the bounding box coordinates
x1, y1, x2, y2 = xyxy[0]  # Assuming only one barrel is detected

# Calculate the center point (2D position)
center_x = (x1 + x2) / 2
center_y = (y1 + y2) / 2

print(f"Barrel 2D Position: ({center_x:.2f}, {center_y:.2f})")

# Calculate the width and height of the bounding box
width = x2 - x1
height = y2 - y1

# Estimate the orientation based on the aspect ratio
aspect_ratio = width / height
if aspect_ratio > 1:
    orientation = 0  # Horizontal
else:
    orientation = 90  # Vertical

print(f"Barrel Orientation (Aspect Ratio): {orientation}°")

# Refine the orientation estimate using ellipse fitting
# Load the image
image = cv2.imread(image_path)

# Extract the barrel region from the bounding box
barrel_region = image[int(y1):int(y2), int(x1):int(x2)]

# Convert to grayscale and apply thresholding
gray = cv2.cvtColor(barrel_region, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

Barrel 2D Position: (129.90, 347.40)
Barrel Orientation (Aspect Ratio): 0°
